### 포털사이트 크롤링
- 소스 얻어오기(네트워크 패키지)
- 소스에서 특정 문자열(수집하고자하는 data)을 지칭하는 선택자 얻기
    - 크롬 개발자도구 사용
- 전체 코드에서 수집하려고 하는 데이터(태그)의 위치를 찾고
    - 태그 파싱 후 데이터 추출

In [26]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [28]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
import pandas as pd
import bs4
import requests
from urllib.request import urlopen

In [7]:
# 네이트 사이트의 기본 메뉴 텍스트 추출
# 메뉴명, 메뉴 링크 추출
url = 'https://www.nate.com/'
# 네이트 서버에 요청 후 응답 반환
html = urlopen(url)
html

# 소스코드 확인
html_text = html.read() # HTTPResponse 객체에 담겨있는 html코드 반환
html_text  # 바이너리 문자열 반환(한글 변경 후 사용 -bs4 객체 사용하면 된다)

b'\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n\t\r\n<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\r\n<meta name="msapplication-starturl" content="//www.nate.com/" />\r\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\r\n<meta name="nate:title" content="" />\r\n<meta name="nate:description" content="\xeb\x84\xa4\xec\x9d\xb4\xed\x8a\xb8 \xec\x9d\xb4\xec\x8a\x88UP" />\r\n<meta name="nate:site_name" content="\xeb\x84\xa4\xec\x9d\xb4\xed\x8a\xb8 \xed\x99\x88" />\r\n<meta name="nate:url" content="https://www.nate.com/" />\r\n<meta name="nate:image" content="" />\r\n<meta name="description" content="\xec\x83\x88\xeb\xa1\x9c\xec\x9b\x8c\xec\xa7\x84 nate\xec\x97\x90\xec\x84\x9c \xeb\x8b\xb9\xec\x8b\xa0\xec\x9d\x98 \xec\x98\xa4\xeb\x8a\x98\xec\x9d\x84 \xeb\xa7\x8c\xeb\x82\x98\xeb\xb3\xb4\xec\x84\xb8\xec\x9a\x94"/>\r\n<meta property="og:title" content="\xed\x95\x9c\xeb\x88\x88\xec\x97\x90 \xeb\xb3\xb4\xeb\x8a\x94 \xec\x98\xa4\xeb\x8a\x98 : \xeb\x84\xa4

In [8]:
# bs4 객체 만들기
bs_obj = bs4.BeautifulSoup(html_text, 'html.parser')
bs_obj  # 정적 크롤링 가능한 코드가 전달 (menu 관련 태그가 있는것을 눈으로 확인해서 판단함)


<!DOCTYPE html>

<html lang="ko">
<head>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="//www.nate.com/" name="msapplication-starturl"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="" name="nate:title"/>
<meta content="네이트 이슈UP" name="nate:description"/>
<meta content="네이트 홈" name="nate:site_name"/>
<meta content="https://www.nate.com/" name="nate:url"/>
<meta content="" name="nate:image"/>
<meta content="새로워진 nate에서 당신의 오늘을 만나보세요" name="description"/>
<meta content="한눈에 보는 오늘 : 네이트" property="og:title"/>
<meta content="https://www.nate.com/" property="og:url"/>
<meta content="https://main.nateimg.co.kr/img/v7/OpenGraphTag_nate_240x240.png" property="og:image"/>
<meta content="새로워진 nate에서 당신의 오늘을 만나보세요" property="og:description"/>
<title>한눈에 보는 오늘 : 네이트</title>
<link href="/css/common.min.css?v=202305031001_01" rel="stylesheet" type="text/css"/>
<link href="//main.nateimg.co.kr/img/v7/favicon_32.ico" rel="shortcut icon" typ

In [12]:
# divGnb 태그 안에 원하는 내용이 들어있다. nata 사이트 상단 메뉴 바
#divGnb  => copy selector
menu = bs_obj.find('div', {'id':'divGnb'})
menu_li = menu.findAll('li')
menu_li
len(menu_li)  # 29개의 li를 찾음

[<li class="mail"><a href="https://mail3.nate.com/#index" onmousedown="nc('NGB01');">메일</a></li>,
 <li class="news"><a href="//news.nate.com/" onmousedown="nc('NEV31');">뉴스</a></li>,
 <li class="pann"><a href="https://pann.nate.com/" onmousedown="nc('NGB02');">판</a></li>,
 <li class="chat1"><a href="https://m.nate.com/aichat.html" onmousedown="nc('NGB51');" target="_blank">AI챗</a><em class="beta">beta</em></li>,
 <li class="tv"><a href="https://tv.nate.com/" onmousedown="nc('NGB43');">TV</a></li>,
 <li class="comics"><a href="https://toonnbook.nate.com/" onmousedown="nc('NGB44');">툰앤북</a></li>,
 <li class="fortune"><a href="https://fortune.nate.com/home/main.nate" onmousedown="nc('NGB05');">운세</a></li>,
 <li class="game"><a href="http://game.nate.com/" onmousedown="nc('NGB06');">게임</a></li>,
 <li class="shopping"><a href="//shopping.nate.com/" onmousedown="nc('NGB17');">쇼핑</a></li>,
 <li class="car"><a href="https://auto.nate.com/" onmousedown="nc('NGB03');">자동차</a></li>,
 <li class="t

29

In [13]:
# 29개의 li에 대해서 텍스트와 a태그의 href 속성값을 확인
# 원하는 데이터만 있는지 확인
for li in menu_li : 
    print(li.text, li.a['href'])

메일 https://mail3.nate.com/#index
뉴스 //news.nate.com/
판 https://pann.nate.com/
AI챗beta https://m.nate.com/aichat.html
TV https://tv.nate.com/
툰앤북 https://toonnbook.nate.com/
운세 https://fortune.nate.com/home/main.nate
게임 http://game.nate.com/
쇼핑 //shopping.nate.com/
자동차 https://auto.nate.com/
팀룸 https://nateonweb.nate.com/teamroom/
네이트뷰 https://view.nate.com/

더보기


뉴스
날씨
스포츠
연예
아이돌24
랭킹뉴스


판
톡톡
판포토
팬톡


네이트온
네이트캐쉬
문자메시지
주소록


컬러링
이슈UP추천

서비스 전체보기
더보기 닫기

 javascript:;
뉴스 //news.nate.com
날씨 //news.nate.com/Weather
스포츠 //sports.news.nate.com/
연예 //news.nate.com/ent/index
아이돌24 //news.nate.com/ent/idol24
랭킹뉴스 //news.nate.com/rank/interest
판 https://pann.nate.com/
톡톡 https://pann.nate.com/talk
판포토 https://pann.nate.com/talk/imageTheme/index
팬톡 https://pann.nate.com/fantalk
네이트온 //nateonweb.nate.com/
네이트캐쉬 //cash.nate.com/center/cashMain.sc
문자메시지 https://sms.nate.com/
주소록 https://mail.nate.com/pims/
컬러링 //mobile.nate.com/
이슈UP추천 //editor.nate.com/


In [15]:
# 29개의 li에 대해서 텍스트와 a태그의 href 속성값을 확인
# 원하는 데이터만 있는지 확인
for li in menu_li : 
    menu_text = li.text 
    menu_link = li.a['href']
    if menu_link == 'javascript:;':
        continue
    print(menu_text, menu_link)    

메일 https://mail3.nate.com/#index
뉴스 //news.nate.com/
판 https://pann.nate.com/
AI챗beta https://m.nate.com/aichat.html
TV https://tv.nate.com/
툰앤북 https://toonnbook.nate.com/
운세 https://fortune.nate.com/home/main.nate
게임 http://game.nate.com/
쇼핑 //shopping.nate.com/
자동차 https://auto.nate.com/
팀룸 https://nateonweb.nate.com/teamroom/
네이트뷰 https://view.nate.com/
뉴스 //news.nate.com
날씨 //news.nate.com/Weather
스포츠 //sports.news.nate.com/
연예 //news.nate.com/ent/index
아이돌24 //news.nate.com/ent/idol24
랭킹뉴스 //news.nate.com/rank/interest
판 https://pann.nate.com/
톡톡 https://pann.nate.com/talk
판포토 https://pann.nate.com/talk/imageTheme/index
팬톡 https://pann.nate.com/fantalk
네이트온 //nateonweb.nate.com/
네이트캐쉬 //cash.nate.com/center/cashMain.sc
문자메시지 https://sms.nate.com/
주소록 https://mail.nate.com/pims/
컬러링 //mobile.nate.com/
이슈UP추천 //editor.nate.com/


In [30]:
# 추출된 메뉴에서 중복 제거 후 df로 생성
# 빈 df 생성
nate_menu = pd.DataFrame({'메뉴':[], '링크':[]})
nate_menu

,메뉴,링크


In [31]:
index = 0
for li in menu_li : 
    menu_text = li.text 
    menu_link = li.a['href']
    if menu_link == 'javascript:;' or menu_text in nate_menu['메뉴'].values:  # .values 를 해야 컬럼안에 있는 값들과 비교한다.
        continue
    temp = pd.DataFrame({'메뉴':menu_text, '링크':menu_link}, index=[index])
    nate_menu = pd.concat([nate_menu, temp])
    index += 1

In [32]:
nate_menu

,메뉴,링크
0,메일,https://mail3.nate.com/#index
1,뉴스,//news.nate.com/
2,판,https://pann.nate.com/
3,AI챗beta,https://m.nate.com/aichat.html
4,TV,https://tv.nate.com/
5,툰앤북,https://toonnbook.nate.com/
6,운세,https://fortune.nate.com/home/main.nate
7,게임,http://game.nate.com/
8,쇼핑,//shopping.nate.com/
9,자동차,https://auto.nate.com/


In [33]:
# 수집 데이터 파일로 저장
# nate_menu.to_csv('./crawl_data/nate_menu.csv')

### 네이버 뉴스 섹션 제목과 링크 크롤링
- 네이버 뉴스는 네이버 정책에 따라 모든 언론사들의 뉴스가 랜덤하게 배치됨
    - 단, 로그인 후 구독을 추가하면 구독한 언론사들의 뉴스가 나옴

In [1]:
import pandas as pd
import bs4
import requests
from urllib.request import urlopen

In [2]:
url ='https://news.naver.com/'
html = urlopen(url)
bs_obj = bs4.BeautifulSoup(html, 'html.parser')

In [3]:
bs_obj


<!DOCTYPE html>

<html lang="ko">
<head>
<title id="browserTitleArea">네이버 뉴스</title>
<script>
	function isMobileDevice() {
		return /^.*(iPhone|iPod|iPad|Android).*/.test(navigator.userAgent);
	}
</script>
<script>
	(function () {
		try {
			if (isMobileDevice() && isAbleApplyPrefersColorScheme()) {
				
				document.querySelector("html").classList.add("DARK_THEME");
			}
		} catch(e) {}

		function isAbleApplyPrefersColorScheme() {
			
			if (window.matchMedia("(prefers-color-scheme)").matches === false) {
				return false;
			}

			var userAgent = navigator.userAgent;

			if (userAgent.indexOf("NAVER") > -1) {
				
				if (/.*NAVER\([a-zA-Z]*;\s[a-zA-Z]*;\s([0-9]*);/.test(userAgent)) {
					return Number(RegExp.$1) >= 1000;
				}
			} else {
				
				return document.cookie.indexOf("NSCS=1") > -1;
			}

			return false;
		}
	})();
</script>
<script>
		var g_ssc = 'news.v3_media' || null;
		</script>
<meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1.0,maximum-s

In [10]:
# 뉴스 섹션 전체를 포함하는 ul 태그 추출
# class - Nlnb_menu_list
uls = bs_obj.findAll('ul',{'class':'Nlnb_menu_list'})
len(uls) # 원소가 1개이므로

ul = bs_obj.find('ul',{'class':'Nlnb_menu_list'})
type(ul)

bs4.element.Tag

In [19]:
ul.findAll('li')[0].find('a')  # 언론사별은 main페이지고 headline 기사가 없음 제외

<a aria-selected="true" class="Nitem_link" href="https://news.naver.com/?viewType=pc" onclick="nclk(event,'lnb.pcmedia','','');" role="menuitem"><span class="Nitem_link_menu">언론사별</span></a>

In [18]:
lis = ul.findAll('li')
# lis[1:]

In [20]:
for li in lis[1:]:
    a_tag = li.find('a')
    print(a_tag.text, " : ", a_tag['href'])

정치  :  https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=100
경제  :  https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101
사회  :  https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=102
생활/문화  :  https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=103
IT/과학  :  https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=105
세계  :  https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=104
랭킹  :  https://news.naver.com/main/ranking/popularDay.naver
신문보기  :  https://news.naver.com/newspaper/home?viewType=pc
오피니언  :  https://news.naver.com/opinion/home
TV  :  https://news.naver.com/main/tv/index.naver?mid=tvh
팩트체크  :  https://news.naver.com/main/factcheck/main.naver
알고리즘 안내  :  https://media.naver.com/algorithm


In [21]:
# 위 메뉴 데이터 df 로 저장
section = []
link = []

In [22]:
for li in lis[1:]:
    a_tag = li.find('a')
    section.append(a_tag.text)
    link.append(a_tag['href'])

In [27]:
len(section)
len(link)

12

12

In [29]:
col_dict = {'section': section, 'link': link}
news_section_df = pd.DataFrame(col_dict)
news_section_df

,section,link
0,정치,https://news.naver.com/main/main.naver?mode=LS...
1,경제,https://news.naver.com/main/main.naver?mode=LS...
2,사회,https://news.naver.com/main/main.naver?mode=LS...
3,생활/문화,https://news.naver.com/main/main.naver?mode=LS...
4,IT/과학,https://news.naver.com/main/main.naver?mode=LS...
5,세계,https://news.naver.com/main/main.naver?mode=LS...
6,랭킹,https://news.naver.com/main/ranking/popularDay...
7,신문보기,https://news.naver.com/newspaper/home?viewType=pc
8,오피니언,https://news.naver.com/opinion/home
9,TV,https://news.naver.com/main/tv/index.naver?mid...


In [30]:
# news_section_df.to_csv('./crawl_data/naver_news_section.csv')